In [4]:
from pathlib import Path
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *
from cmd_classes_funcs_Marchese import MyModel, ImageWithCmdDataset, get_class_labels, get_filenames

In [5]:
# Get classes and filenames
path = Path("data")
classes = get_class_labels(path)
all_filenames = get_filenames(path)

In [6]:
# Get dataset
dataset = ImageWithCmdDataset(classes, all_filenames)

## Splitting Data into train/validate sets

In [7]:
# Getting size of dataset and corresponding list of indices
dataset_size = len(dataset.all_filenames)
dataset_indices = list(range(dataset_size))

In [8]:
# Shuffling the indices
np.random.shuffle(dataset_indices)

In [9]:
# Getting index for where we want to split the data
val_split_index = int(np.floor(0.2 * dataset_size))

In [10]:
# Splitting list of indices into training and validation indices
train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

In [11]:
# Creating samplers
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [12]:
train_loader = DataLoader(dataset=dataset, shuffle=False, batch_size=16, sampler=train_sampler)
val_loader = DataLoader(dataset=dataset, shuffle=False, batch_size=16, sampler=val_sampler)

In [13]:
# Instantiate MyModel class
net = MyModel()

In [23]:
# defining loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr =0.002)

In [24]:
num_epochs = 50

In [25]:
# Model training
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for data in train_loader:
        # get the inputs
        inp_data, label = data

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        output = net(inp_data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print(f"Training loss at epoch {epoch}/{num_epochs}:" + str(running_loss))

print('Finished Training')

Training loss at epoch 0/50:734407.6653251648
Training loss at epoch 1/50:38626.86206626892
Training loss at epoch 2/50:28604.067169189453
Training loss at epoch 3/50:10575.522519011425
Training loss at epoch 4/50:5100.560373734683
Training loss at epoch 5/50:2962.457620885671
Training loss at epoch 6/50:2171.4828407882014
Training loss at epoch 7/50:2387.4256314779973
Training loss at epoch 8/50:1190.645436181659
Training loss at epoch 9/50:930.0566432487838
Training loss at epoch 10/50:850.0747101508053
Training loss at epoch 11/50:1224.7386995327033
Training loss at epoch 12/50:1535.0129023486907
Training loss at epoch 13/50:657.9374317722884
Training loss at epoch 14/50:290.05102610344227
Training loss at epoch 15/50:185.96324007109365
Training loss at epoch 16/50:146.19248726676915
Training loss at epoch 17/50:126.93180081435446
Training loss at epoch 18/50:119.2474887117396
Training loss at epoch 19/50:96.96433655175849
Training loss at epoch 20/50:81.12622258849728
Training loss

In [27]:
# Checking accuracy on valisation set

correct = 0
total = 0

with torch.no_grad():

    for data in val_loader:

        inp_data, label = data

        # predict
        output = net(inp_data)
        
        # Checking if there are multiple batches
        if output.size()[0] > 1:
            
            # For-loop accounts for multiple batches
            for i in range(output.size()[0]):
                
                # Getting the probability of the predicted most probable move
                prob = torch.max(output[i])
                move = int()
                
                # Looking at what move has that probability
                if prob == output[i][0]:
                    move = 0
                elif move == output[i][1]:
                    move = 1
                else:
                    move = 2
                
                # Checking if the predicted move is the same as the label
                if move == label[i]:
                    correct +=1
                total +=1
                
        else:
            # Getting the probability of the predicted most probable move
            prob = torch.max(output)
            move = int()
            
            # Looking at what move has that probability
            if prob == output[0]:
                move = 0
            elif move == output[1]:
                move = 1
            else:
                move = 2
                
            # Checking if the predicted move is the same as the label
            if move == label:
                correct +=1
            total +=1
            
        
accuracy = correct / total
print(f"Accuracy on validation set: {correct}/{total} = {accuracy*100:.2f}%")

Accuracy on validation set: 897/1135 = 79.03%


In [26]:
PATH = 'cmd_torch.pth'
torch.save(net.state_dict(), PATH)